# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
DataFrame = pd.read_csv("wheaters.csv")
DataFrame.dropna()
DataFrame.head()



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,banda aceh,0,IN,1619574736,5,25.4833,80.3333,92.59,9.44
1,pawai,0,IN,1619574736,6,24.2667,80.1667,90.23,4.83
2,bhanpuri,5,IN,1619574853,8,21.1000,80.9167,90.68,1.79
3,bharatpur,0,IN,1619574855,8,27.2167,77.4833,91.15,4.74
4,bokoro,0,TD,1619574887,9,12.3807,17.0568,83.12,12.73


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Config Gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use 'Lat' and 'Lng' as the weight.
locations = DataFrame[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = DataFrame["Humidity"].astype(float)


In [5]:
locations
humidity

0        5.0
1        6.0
2        8.0
3        8.0
4        9.0
       ...  
539    100.0
540    100.0
541    100.0
542    100.0
543    100.0
Name: Humidity, Length: 544, dtype: float64

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=10,
                                   point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

new_df = pd.DataFrame(DataFrame, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

# A max temperature lower than 80 degrees but higher than 70.

max_temp = (new_df["Max Temp"] <= 80) & (new_df["Max Temp"] > 70)

# Wind speed less than 10 mph.

wind_speed = new_df["Wind Speed"] < 10

# Zero cloudiness.

cloudiness = new_df["Cloudiness"] == 0

# Drop any rows that don't contain all three conditions. 


# New dataframe [max_temp & wind_speed & cloudiness]
new_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
5,abeche,78.26,8.81,0
65,carnarvon,78.80,8.05,0
87,mecca,79.72,8.48,0
91,shahganj,77.00,3.44,0
132,coxim,71.38,5.01,0
135,guerrero negro,74.25,8.34,0
240,saint-pierre,77.00,4.61,0
241,saint-joseph,77.00,4.61,0
243,saint-leu,77.00,4.61,0
244,saint-philippe,77.00,4.61,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame(DataFrame, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

# hotel_df
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,banda aceh,IN,25.4833,80.3333,
1,pawai,IN,24.2667,80.1667,
2,bhanpuri,IN,21.1000,80.9167,
3,bharatpur,IN,27.2167,77.4833,
4,bokoro,TD,12.3807,17.0568,
...,...,...,...,...,...
539,byron bay,US,44.0327,-92.6455,
540,cape town,CL,-36.4130,-71.9028,
541,esperance,AU,-33.8667,121.9000,
542,grindavik,IS,63.8424,-22.4338,


In [10]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# print("A HOTEL WITHIN A 5000 METER RADIUS")

# response = requests.get(base_url, params)
# # print(response.url)

# hotel_distance = response.json()
# print(json.dumps(hotel_distance, indent=4, sort_keys=True))

target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [11]:

# # params
pprint(response.json(), depth=1)

{'html_attributions': [], 'results': [...], 'status': 'OK'}


In [12]:
hotel_df.iloc[0][0]

'banda aceh'

In [13]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary 
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()

    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [14]:
hotel_name


['Hotel Sarang Intercontinental',
 'Vardhaman Laaj',
 'Aqua Village Water Park And Resort',
 'The Bagh',
 nan,
 'Hotel Cesar',
 'Hotel Godhuli',
 nan,
 'SHRI GURU SHARNAM PALACE',
 'Country Inn & Suites by Radisson Kota',
 nan,
 'Tianma Hotel',
 'Hilton Garden Inn Sacramento Elk Grove',
 nan,
 nan,
 'Courtyard by Marriott Medford Airport',
 'AC Hotel by Marriott Guadalajara, Mexico',
 nan,
 'Saucillo Inn',
 'Jinchuan Hotel',
 'Best Western Plus Chena River Lodge',
 'Latitude20 Roebourne Village',
 nan,
 nan,
 'Hotel Barun Sankhuwasabha',
 'Erenhot Weihao Hotel',
 nan,
 nan,
 'Sea Hotel',
 'Grand View Inn & Suites',
 nan,
 'نزل البحر والصحراء',
 'Bishapour Hotel',
 'Steppe hotel',
 'The Garden Hotel Korla',
 'Raoum Inn Hotel',
 'Ghaem Hotel',
 'la vieille Maison Rouge',
 'Hotel Shere Punjab Regency',
 'Hotel Shere Punjab Regency',
 'Damei Grand New Century Hotel',
 'The Kanata Inns - Blairmore Hotel',
 'Diyor hotel',
 'Hotel Nelligan',
 'Al Diyar Hotel',
 'Hôtel Teriya',
 'Hôtel Teriya'

In [14]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()


,City,Country,Lat,Lng,Hotel Name
0,banda aceh,IN,25.4833,80.3333,Hotel Sarang Intercontinental
1,pawai,IN,24.2667,80.1667,Vardhaman Laaj
2,bhanpuri,IN,21.1000,80.9167,Aqua Village Water Park And Resort
3,bharatpur,IN,27.2167,77.4833,The Bagh
5,abeche,TD,13.8292,20.8324,Hotel Cesar
...,...,...,...,...,...
537,syracuse,IT,37.0881,15.2792,Caportigia Boutique Hotel
540,cape town,CL,-36.4130,-71.9028,Hotel Molino Viejo
541,esperance,AU,-33.8667,121.9000,Comfort Inn Bay Of Isles
542,grindavik,IS,63.8424,-22.4338,Northern Light Inn & Max's Restaurant


In [18]:
hotel_df.to_csv('hotel_exp.csv')

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

markers = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))